In [31]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize

"""
The following is a simple constrained minimization problem.

The scipy.optimize.minimize function can take equality and inequality constraints.

The equality and inequality constraints are represented by f(x)=0 and f(x)>0, respectively.

obj: x**2+y

s.t.

2*x - y = 0

x + y + 1 > 0

"""

def objective(X):
    x, y = X
    
    return x**2 + y

def eq(X):
    x, y = X
    return 2 * x - y

def ineq(X):
    x, y = X
    return x + y + 1

sol = minimize(objective, [1, -0.5], constraints=({'type': 'eq', 'fun': eq},
                                                  {'type': 'ineq', 'fun': ineq}))

sol

     fun: -0.5555555555555555
     jac: array([-0.66666665,  1.        ])
 message: 'Optimization terminated successfully.'
    nfev: 8
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([-0.33333333, -0.66666667])

In [34]:
"""
Scipy uses SLSQP as the default solver for constrained optimization. 

Genetic optimization utilizes chaotic mutations which require a smooth loss function.

To accomplish this an L2 penalty (P1 & P2) is applied when the constraints are not met.

P1 * equality_constraint(x)**2 + P2 * min(0,inequality_constraint(x))**2

"""

def smooth_objective(X):
    x, y = X
    
    return (x**2 + y) + 10*eq(X)**2 + 10*min(0,ineq(X))**2

In [58]:
import numpy as np

"""
The search bound for the 1st generation can be defined explictly.
"""

pop_size = 10001
bounds = ((-5,5),(-5,5))

#actual solution
SOL = sol.x


X = [np.random.uniform(l,u,size=(1,101)) for l,u in bounds]
X = np.vstack(X)

In [59]:
print(np.min(L))

-0.5682061225212588


In [60]:
"""
The best parameters are sorted from smallest to largest.

Should half of the population be removed?

Then the population is split into 1...n (best) and n+1...m (other)
"""

L = np.array([smooth_objective(X[:,i]) for i in range(X.shape[1])])

shift = np.argsort(L)

L = L[shift]
X = X[:,shift]

X = X[:,:X.shape[1] - X.shape[1]%2]

X_N = X[:,:X.shape[1]//2] 
X_M = X[:,X.shape[1]//2:]

In [61]:
from math import gamma

In [64]:
DATA = []
for _ in range(10):
    Y_N = np.zeros((X.shape[0],X.shape[1]//2))
    Y_M = np.zeros((X.shape[0],X.shape[1]//2))
    Y_K = np.zeros((X.shape[0],X.shape[1]//2))
    Y_L = np.zeros((X.shape[0],X.shape[1]//2))
    MUT = np.zeros((X.shape[0],X.shape[1]//2))

    BEST = X[:,0] 
    
    DATA.append(X[:,0:1])

    for i in range(X.shape[1]//2):
        
        """
        The distribution based crossovers are created with directional components
        (1) - an average of the top N samples, TOP_N_AVG
        (2) - the current best sample, BEST
        (3) - an average of (1),(2), and the ith sample of the top N samples, X_N[:,i]
        
        (2) defined the center of a normal distribution crossover operation.
        The variation around this point is defined by the uniform variation between (1) and (2)
        
        (3) defined the center of a normal distribution crossover operation.
        The variation around this point is defined by the uniform variation between
        X_N[:,i] and X_M[:,i].
        
        (2) defined the initial point of a uniform distribution crossover.
        The point is shifted in a randomly in the direction between X_N[:,i]-X_M[:,i]
        
        (3) defined the initial point of a uniform distribution crossover.
        The point is shifted in a randomly in the direction between (2)-(3)
        
        
        """

        TOP_N_AVG = np.mean(X_N,axis=1)

        DIRECTION_AVG = 1./3. * (TOP_N_AVG+BEST+X_N[:,i])

        DIRECTION_DEV = ((X_N[:,i] - X_M[:,i])/12)**2 + 1e-9

        BEST_DEV = ((BEST-TOP_N_AVG)/12)**2 + 1e-9

        Y_N[:,i] = [np.random.normal(DIRECTION_AVG[j],DIRECTION_DEV[j]) for j in range(X.shape[0])]

        Y_M[:,i] = [np.random.normal(BEST[j],BEST_DEV[j]) for j in range(X.shape[0])]
        
        Y_K[:,i] = BEST + np.random.uniform(0,1)*(X_N[:,i]-X_M[:,i])
        
        Y_L[:,i] = DIRECTION_AVG + np.random.uniform(0,1)*(BEST-DIRECTION_AVG)

        """
        Different mutation operators are used depending on the iteration
        1 - Cauchy mutation operator
        2 - normal mutation operator
        3 - levy flight mutation operator https://arxiv.org/pdf/1303.6342.pdf
        """
        
        if i % 3 == 0:
            Z_CAUCHY = X_N[:,i]+ X_N[:,i]*np.random.standard_cauchy(size=(X.shape[0]))
            MUT[:,i] = Z_CAUCHY
        elif i % 3 == 1:
            NORM_MUT_DEV = ((BEST-X_N[:,i])/12)**2 + 1e-9
            Z_NORM = np.random.normal(BEST,NORM_MUT_DEV,size=(X.shape[0]))
            MUT[:,i] = Z_NORM
        elif i % 3 == 2:
            lmbda = 1
            LEVY_DEV = (gamma(1+lmbda)*np.sin(np.pi*lmbda/2)/(gamma((1+lmbda)/2)*lmbda*2**((lmbda-1)/2)))**(1/lmbda)
            LEVY_U = np.random.normal(0,LEVY_DEV,size=(X.shape[0]))
            LEVY_V = np.random.normal(0,1,size=(X.shape[0]))
            Z_LEVY = X_N[:,i]+ 0.01*LEVY_U/(LEVY_V**(1/lmbda))
            MUT[:,i] = Z_LEVY

    X = np.hstack([X_N,Y_N,Y_M,Y_K,Y_L,MUT])
    
    #drop duplicates (substitution and substitute with random)

    NUM = X.shape[1]
    
    X = np.unique(X, axis=1)
    
    if NUM-X.shape[1]>0:
        sample_size = NUM-X.shape[1]
        RAND = [np.random.uniform(l,u,size=(1,101)) for l,u in bounds]
        RAND = np.vstack(RAND)
        X = np.hstack([X,RAND])
    

    L = np.array([smooth_objective(X[:,i]) for i in range(X.shape[1])])

    shift = np.argsort(L)

    L = L[shift]
    X = X[:,shift]
    X = X[:,:1000]

    X = X[:,:X.shape[1] - X.shape[1]%2]

    X_N = X[:,:X.shape[1]//2] 
    X_M = X[:,X.shape[1]//2:]

    print('iteration', _, 'minimum loss',np.min(L))



iteration 0 minimum loss -0.5682065217328194
iteration 1 minimum loss -0.5682065217362197
iteration 2 minimum loss -0.5682065217380334
iteration 3 minimum loss -0.5682065217382758
iteration 4 minimum loss -0.5682065217384538
iteration 5 minimum loss -0.5682065217385865
iteration 6 minimum loss -0.5682065217386401
iteration 7 minimum loss -0.5682065217386854
iteration 8 minimum loss -0.5682065217387151
iteration 9 minimum loss -0.5682065217387383


In [65]:
gen_sol = X[:,0]
gen_sol

array([-0.33152165, -0.69076081])